# 케라스를 이용한 회귀

In [ ]:
from keras.models import Sequential # Sequential은 models의 서브패키지
from keras.layers import Dense # Dense는 layers의 서브패키지   => layers에서 모델구성
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler # 신경망에서의 정규화

X, y = make_regression( n_samples=100, n_features=2, noise=0.1, random_state=1 ) # 100x2,  100    # 관측치 100, 변수 2
scalarX , scalarY = MinMaxScaler(), MinMaxScaler() # 왜 2개 만드나? 데이터사이즈 즉 값의 범위가 다르니까

scalarX.fit(X)
scalarY.fit( y.reshape(100, 1) ) # 열로 만들어져있으니 행으로 만들어줘서 fit하자

X = scalarX.transform(X)
y = scalarY.transform(y.reshape(100,1))

# keras model : 전레이어의 결과가 다음의 레이어에 자동으로 입력                  # FFNN망 만들떄 dence씀
model = Sequential()
# 첫번째 레이어 : 100x2 입력되어 덴스4(출력차수지정) 즉 가중치가 2x4로 되어 => 출력형태는 100x4
model.add( Dense(4, input_dim=2, activation='relu') )
# 두번째 레이어 : 100x4 -> 4x4 => 100x4
model.add( Dense(4, activation='relu') )
# 세번쨰 레이어 : 100x4 -> 4x1 => 100x1    # 100개의 관측치로부터 하나의 결과를 즉, 예측!
model.add( Dense(1, activation='linear') ) # 선형? 예측!
model.compile( loss='mse', optimizer='adam' ) # 텐서플로 모델로 변환(컴파일;)
# loss 예) 예측 mse, 분류 cross enthropy, GAN KL-divergence:분포를 비교
model.fit(X, y, epochs=1000, verbose=0) # 텐서플로에서의 에포크 포문과 미니배치 포문

Xnew, a = make_regression( n_samples=3, n_features=2, noise=0.1, random_state=1 )
Xnew = scalarX.transform(Xnew)
ynew = model.predict(Xnew)

for i in range(len(Xnew)):
    print("입력데이터=%s, 예측결과=%s" % (Xnew[i], ynew[i]))

# 케라스를 이용한 분류

In [ ]:
from keras.models import Sequential # 시퀀셜 : 입력이 하나, 출력도 하나인 망
from keras.layers import Dense
import numpy

numpy.random.seed(7)
dataset = numpy.loadtxt("pima.data", delimiter=",")
X = dataset[:, 0:8] # 독립변수 8개
Y = dataset[:, 8] # 종속변수 1개

model = Sequential()
# 첫번째 레이어 : ? x 8 -> 8x12 => ?x12
model.add( Dense(12, input_dim=8, activation="relu") )
# 두번째 레이어 : ?x12 -> 12x8 -> ?x8
model.add( Dense(8, activation="relu") )
# 세번째 레이어 : ?x8 -> 8x1 => ?x1
model.add( Dense(1, activation="sigmoid") ) # 시그모이드? 0.5를 기준으로 분류
model.compile( loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"] ) # 메트릭스 accuracy를 보고싶다(측정하겠다;)         
model.fit(X, Y, epochs=150, batch_size=10)

scores = model.evaluate(X ,Y) # 평가해보자
print( "\n%s: %.2f%%" % ( model.metrics_names[1], scores[1]*100 ) ) # accuracy 보려면 컴파일할때 적어줘야함

In [ ]:
# 케라스 분류 예2)
from keras.models import Sequential
from keras.layers import Dense
import numpy

numpy.random.seed(7)
dataset = numpy.loadtxt("pima.data", delimiter=",")
X = dataset[:, 0:8]
Y = dataset[:, 8]

model = Sequential()

model.add( Dense(12, input_dim=8, kernel_initializer='uniform', activation="relu") ) # kernel_initializer 가중치 초기화 uniform 균등분포화로 초기화하

model.add( Dense(8, kernel_initializer='uniform', activation="relu") )

model.add( Dense(1, kernel_initializer='uniform', activation="sigmoid") )
model.compile( loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"] )

# 데이터 분리 입력 시(train, test) 과적합이 되는 경우도
# 그래서 train, validation, test로 나눠서 함
history = model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10)

scores = model.evaluate(X ,Y)
print( "\n%s: %.2f%%" % ( model.metrics_names[1], scores[1]*100 ) )

In [ ]:
# 로스, 정확도 확인
import matplotlib.pyplot as plt
%matplotlib inline

print( history.history.keys() ) # dict_keys(['val_loss', 'val_acc', 'loss', 'acc']) 4개가 나옴

plt.figure( figsize = (10,10) )

plt.subplot(221)
plt.plot( history.history['acc'] )
plt.plot( history.history['val_acc'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend( ['train', 'test'], loc='upper left')

plt.subplot(222)
plt.plot( history.history['loss'] )
plt.plot( history.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend( ['train', 'test'], loc='upper left')

plt.show()

# 비선형으로 바이너리분류 해보자

In [ ]:
from sklearn.preprocessing import scale
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, Y = make_moons(noise=0.2, random_state=0, n_samples=1000)
X = scale(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.5)

print(X.shape)
print(Y.shape)

In [ ]:
fig, ax = plt.subplots()
ax.scatter( X[ Y==0, 0 ], X [ Y==0, 1 ], label="Class 0")
ax.scatter( X[ Y==1, 0 ], X [ Y==1, 1 ], color="r", label="Class 1")
ax.legend()
ax.set(xlabel="X", ylabel="Y", title="binary classification")

In [ ]:
# 비선형 모델 해결? -> 신경망으로 한다!
import keras

model = Sequential()
# 행렬연산
# 차원확대 -> 설명을 확대 즉, 변수를 32개가 늘어난것과 마찬가지
# 변수가 늘어나면 설명력이 상승 세밀하게 설명된다는 의미
# 1000x2 2x32[64개 + 바이어스 32개]1000x32
model.add( Dense(32, input_dim=2, activation="relu") ) # 차원확대시키고있
# 1000x32 32x1[32 + 바이어스 1개]1000x1
model.add( Dense(1, activation="sigmoid") )
model.compile( optimizer="AdaDelta", loss="binary_crossentropy", metrics=["accuracy"])

# callback 함수 : window 자동으로 호출되는 함수
tb_callback = keras.callbacks.TensorBoard( log_dir = "./Graph/model_1", # graph 이미지 출력
                                          histogram_freq=100,
                                          write_graph=True,
                                          write_images=False )
tb_callback.set_model(model) # 콜백을 모델에 등록

In [ ]:
hist = model.fit( X_train, Y_train, batch_size=32, epochs=200,
                 verbose=0, validation_data=(X_test, Y_test),
                 callbacks=[tb_callback] )

In [ ]:
score = model.evaluate( X_test, Y_test, verbose=0 )
print("Test loss : ", score[0] )
print("Test acc : ", score[1])

In [ ]:
keras.utils.print_summary(model) # 모델 구성을 출력

In [ ]:
keras.utils.plot_model(model) # 모델의 구조를 출력

In [ ]:
%matplotlib inline
from keras.utils.vis_utils import model_to_dot
model_to_dot(model)

In [ ]:
from IPython.display import Image, SVG
SVG( model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
model.summary()

# Multi Classification

In [ ]:
# Callback class
# custom callback 함수 제작
# 함수에 의해 호출되는 객체.
class CustomHistory(keras.callbacks.Callback): # 케라스 콜백 상속
    def init(self): # 오버라이딩
        self.train_loss = []
        self.val_loss = []
        self.train_acc = []
        self.val_acc = []
        
    def on_epoch_end( self, batch, logs={} ): # 오버라이딩
        self.train_loss.append( logs.get('loss') )
        self.val_loss.append( logs.get('val_loss') )
        self.train_acc.append( logs.get('acc') )
        self.val_acc.append( logs.get('val_acc') )

In [ ]:
from keras.datasets import mnist
import matplotlib.pylab as plt
(X_train0, y_train0), (X_test0, y_test0) = mnist.load_data()

print(X_train0.shape, X_train0.dtype) # (60000, 28, 28) uint8
print(y_train0.shape, y_train0.dtype) # (60000,) uint8
print(X_test0.shape, X_test0.dtype) # (10000, 28, 28) uint8
print(y_test0.shape, y_test0.dtype) # (10000,) uint8

In [ ]:
plt.imshow(X_train0[0])
plt.grid(False)
plt.show()

In [ ]:
# fully-connected : flatten : FFNN은 1차원(1줄)으로 들어가줘야한다.
# 이미지 정규화 : 0~255 컬러값 / 255 => 0~1

X_train = X_train0.reshape(60000, 784).astype('float32')/255.0
X_test = X_test0.reshape(10000, 784).astype('float32')/255.0
print(X_train.shape, X_train.dtype)

In [ ]:
y_train0[:5] # one-hot-encoding 여부 : 멀티레이블인 경우에는 소프트맥스로..
# 소프트맥스는 경우의수에 대한 확률값을 나타내줌 그렇기 때문에 원핫인코딩해줘야

In [ ]:
# to_categorical : 원핫인코딩해주는 함수
from keras.utils import np_utils

Y_train = np_utils.to_categorical(y_train0, 10)
Y_test = np_utils.to_categorical(y_test0, 10)
Y_train[:5]

In [ ]:
import numpy as np
from keras import optimizers

np.random.seed(0)

model = Sequential()
# 60000x784, 784x15 => 60000x15 
model.add( Dense(15, input_dim=784, activation="sigmoid") )
# 60000x15, 15x10 => 60000x10
model.add( Dense(10, activation="sigmoid") )
# lr : learning rate
# stochastic gradient descent
model.compile( optimizer=optimizers.SGD(lr=0.2), loss="mean_squared_error", metrics=["accuracy"] )

In [ ]:
model.layers # 레이어 출력 및 확인

In [ ]:
l1 = model.layers[0]
l2 = model.layers[1]

# 레이어 속성으로 정보 확인
l1.name
l1.input_shape
l1.output_shape
l1.activation

In [ ]:
custom_hist = CustomHistory()
# 콜백함수 어떻게 쓰는지 보라. 콜백객체있어야하고 모델핏할때 매개변수에 지정도 해줘야 한다.
custom_hist.init()

hist = model.fit( X_train, Y_train, nb_epoch=30, batch_size=100,
                 validation_data=(X_test, Y_test), callbacks=[custom_hist], verbose=2 )

plt.plot( hist.history['loss'] )
plt.show()

In [ ]:
plt.plot(hist.history['acc'], 'b-', label="training")
plt.plot(hist.history['val_acc'], 'r:', label="test")
plt.legend()
plt.show()

print(hist.model)
print(hist.params)
print(hist.history['acc'])
print(hist.history['val_acc'][29])

plt.plot(custom_hist.train_loss, 'y', label="train loss")
plt.show()

In [ ]:
# 문제 X_test의 이미지 한 장의 label을 예측해 보시오

plt.imshow(X_test0[0])
plt.grid(False)
plt.show()

model.predict( X_test[:1, :] ) # 확률로 나오네?

In [ ]:
model.predict_classes(X_test[:1, :], verbose=0)

In [ ]:
# 출력차수 512개

60000x784 784x512 60000x512


# 회귀 해보자

In [ ]:
from keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
model = Sequential()
model.add( Dense( 1, input_dim=13, activation="linear" ) )

In [ ]:
model.summary()
model.compile( optimizer='rmsprop', loss="mse", metrics=["mae"] )
model.fit( x_train, y_train, batch_size=1, epochs=10, verbose=1 )

In [ ]:
mse, mae = model.evaluate( x_test, y_test, verbose=False )
#  리턴값은 컴파일 당시의 loss파라미터와 metrics 파라미터에 대한 값이다

rmse = np.sqrt(mse)
mse, rmse, mae

In [ ]:
# 문제 : x_test에 있는 집 3개를 예측해보시오

In [ ]:
pred = model.predict( x_test[:3, :] ).reshape(1,3)

In [ ]:
real = y_test[:3]

In [ ]:
# 상관계수를 확인하시오!

import numpy as np
np.corrcoef(pred, real)

# scikits와 keras를 연결해주는 객체 : KerasClassifier, KerasRegressor

In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# 768x9  =>  768x8,  768x1
# 변수 8개
def create_model():
    model = Sequential()
    # 가중치 8x12 -> 아웃풋 768x12
    model.add( Dense(12, input_dim=8, activation='relu') )
    # 768x12 -> 12x1 => 768x1 
    model.add( Dense(1, activation='sigmoid') ) # 0~1
    model.compile( loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

seed = 7
numpy.random.seed(seed)
dataset = numpy.loadtxt("pima.data", delimiter=",")
print(dataset.shape)

X = dataset[:, 0:8]
Y = dataset[:, 8]
model = KerasClassifier( build_fn=create_model, verbose=0 ) # verbose : 실행시 메시지 보고싶은지 아닌지 정하는 파람

- gridsearchCV로 좋은 파라미터 조합 찾기

In [ ]:
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
# 요즘엔 이렇게 매개변수입력 형태는 딕셔너리 형태로 넣는다.
param_grid = dict( batch_size=batch_size, epochs=epochs)

grid = GridSearchCV( estimator=model, param_grid=param_grid, n_jobs=-1) # n_jobs : 참여하는 cpu개수
grid_result = grid.fit(X, Y)
print("최적스코어 : %f\t사용한파라미터조합 : %s" % (grid_result.best_score_, grid_result.best_params_) )

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results_["params"]

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with : %r" % (mean, stdev, param))

In [ ]:
# 문제
# Dense1 : activation함수를 달아주고
# [softmax, softplus, softsign, relu, tanh, sigmoid, hard_sigmoid, linear]
# 의 조합 중 가장 좋은 activation 함수를 결정하시오

# Dense2 : 가중치 초기화(kernel_initializer) 매개변수를 넣고 이를
# [uniform, lecun_uniform, normal, zero, glorot_normal, glorot_uniform, he_normal, he_uniform]
# 테스트해서 가장 최적의 가중치 초기화 파라미터를 결정해보자

In [ ]:
# 768x9  =>  768x8,  768x1
# 변수 8개
from keras.layers import Dropout

def create_model(activation='uniform', kernel_initializer='relu', dropout_rate=0.0):
    model = Sequential()
    # 가중치 8x12 -> 아웃풋 768x12
    model.add( Dense(12, input_dim=8, kernel_initializer='uniform', activation=activation) )
    # 768x12 -> 12x1 => 768x1 
    model.add( Dropout(dropout_rate) ) # 과적합방지
    model.add( Dense(1, kernel_initializer=kernel_initializer, activation='sigmoid') ) # 0~1
    model.compile( loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

seed = 7
numpy.random.seed(seed)
dataset = numpy.loadtxt("pima.data", delimiter=",")
print(dataset.shape)

X = dataset[:, 0:8]
Y = dataset[:, 8]

# Early Stopping : 
from keras.callbacks import EarlyStopping
stopper = EarlyStopping( monitor="val_acc", patience=3, verbose=1) # moniter : 확인할 값, patience : 얼마나 참고 기회를 줄지

model = KerasClassifier( build_fn=create_model, batch_size=20, epochs=100, verbose=0 ) # verbose : 실행시 메시지 보고싶은지 아닌지 정하는 파람

In [ ]:


activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
kernel_initializer = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# 요즘엔 이렇게 매개변수입력 형태는 딕셔너리 형태로 넣는다.
param_grid = dict( activation=activation, kernel_initializer=kernel_initializer, dropout_rate=dropout_rate)

grid = GridSearchCV( estimator=model, param_grid=param_grid, n_jobs=-1) # n_jobs : 참여하는 cpu개수
fit_params = dict(callbacks=[stopper])
#변동매개변수(여러개가 전달될 때) list는 * dict는 **

grid_result = grid.fit(X, Y, **fit_params)
print("최적스코어 : %f\t사용한파라미터조합 : %s" % (grid_result.best_score_, grid_result.best_params_) )

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results_["params"]

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with : %r" % (mean, stdev, param))

# Pipeline

In [1]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense
import pandas
import numpy

Using TensorFlow backend.


In [2]:
dataframe = pandas.read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values
X = dataset[:, 0:13]
Y = dataset[:, 13] # reshape

In [3]:
def baseline_model():
    model = Sequential()
    # 506x13 13x13 506x13
    model.add( Dense(13, input_dim=13, kernel_initializer='normal', activation='relu') )
    # 506x13 13x1  506x1
    model.add( Dense(1, kernel_initializer='normal') )
    model.compile( loss='mean_squared_error', optimizer='adam' )
    return model

In [4]:
seed = 7
numpy.random.seed(seed)
estimator1 = KerasRegressor( build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)

In [5]:
# larger_model   베이스보다 1칸더 딥하게
def larger_model():
    model = Sequential()
    model.add( Dense(13, input_dim=13, kernel_initializer='normal', activation='relu') )
    model.add( Dense(6, kernel_initializer='normal', activation='relu') )
    model.add( Dense(1, kernel_initializer='normal') )
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [6]:
seed = 7
numpy.random.seed(seed)
estimator2 = KerasRegressor( build_fn=larger_model, nb_epoch=100, batch_size=5, verbose=0)

In [7]:
# wider_model
def wider_model():
    model = Sequential()
    # 506x13 13x13 506x13
    model.add( Dense(20, input_dim=13, kernel_initializer='normal', activation='relu') )
    # 506x13 13x1  506x1
    model.add( Dense(1, kernel_initializer='normal') )
    model.compile( loss='mean_squared_error', optimizer='adam' )
    return model

In [8]:
seed = 7
numpy.random.seed(seed)
estimator3 = KerasRegressor( build_fn=wider_model, nb_epoch=100, batch_size=5, verbose=0)

In [9]:
kfold = KFold( n_splits=10, random_state=seed) # 10개 중 한개는 valiation으로 사용
results1 = cross_val_score(estimator1, X, Y, cv=kfold)
results2 = cross_val_score(estimator2, X, Y, cv=kfold)
results3 = cross_val_score(estimator3, X, Y, cv=kfold)
print("base Results : %.2f (%.2f) MSE" % ( results1.mean(), results1.std() ) )
print("larger Results : %.2f (%.2f) MSE" % ( results2.mean(), results2.std() ) )
print("wider Results : %.2f (%.2f) MSE" % ( results3.mean(), results3.std() ) )

C:\Users\ICT01_03\Anaconda3\envs\venv\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning



base Results : -115.04 (83.33) MSE
larger Results : -263.17 (310.80) MSE
wider Results : -119.81 (92.09) MSE


In [10]:
numpy.random.seed(seed)
estimators1 = []
estimators2 = []
estimators3 = []

estimators1.append( ('standardize', StandardScaler() ) )
estimators2.append( ('standardize', StandardScaler() ) )
estimators3.append( ('standardize', StandardScaler() ) )

estimators1.append( ('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)) )
estimators2.append( ('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)) )
estimators3.append( ('mlp', KerasRegressor(build_fn=wider_model, epochs=50, batch_size=5, verbose=0)) )


pipeline1 = Pipeline(estimators1) # Pipeline의 매개변수는 리스트로 넣어야 하므로 리스트형태로 estimator를..
pipeline2 = Pipeline(estimators2)
pipeline3 = Pipeline(estimators3)

kfold = KFold(n_splits=10, random_state=seed)
results1 = cross_val_score(pipeline1, X, Y, cv=kfold)
results2 = cross_val_score(pipeline2, X, Y, cv=kfold)
results3 = cross_val_score(pipeline3, X, Y, cv=kfold)

print("base Standardized : %.2f (%.2f) MSE" % ( results1.mean(), results1.std() ) )
print("larger Standardized : %.2f (%.2f) MSE" % ( results2.mean(), results2.std() ) )
print("wider Standardized : %.2f (%.2f) MSE" % ( results3.mean(), results3.std() ) )

base Standardized : -29.45 (27.74) MSE
larger Standardized : -23.08 (26.32) MSE
wider Standardized : -27.66 (27.45) MSE


In [11]:
pipeline1.fit(X,Y)

Pipeline(memory=None,
         steps=[('standardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('mlp',
                 <keras.wrappers.scikit_learn.KerasRegressor object at 0x00000172C2607EB8>)],
         verbose=False)

In [12]:
pipeline2.fit(X,Y)

Pipeline(memory=None,
         steps=[('standardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('mlp',
                 <keras.wrappers.scikit_learn.KerasRegressor object at 0x00000172C2607F60>)],
         verbose=False)

In [13]:
pipeline3.fit(X,Y)

Pipeline(memory=None,
         steps=[('standardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('mlp',
                 <keras.wrappers.scikit_learn.KerasRegressor object at 0x00000172C2607F98>)],
         verbose=False)

In [14]:
res1 = pipeline1.predict(X)
res2 = pipeline2.predict(X)
res3 = pipeline3.predict(X)

In [15]:
# 회귀 평가할 땐 corrcoef
import numpy as np

np.corrcoef(res1, Y)

array([[1.        , 0.90310693],
       [0.90310693, 1.        ]])

In [16]:
np.corrcoef(res2, Y)

array([[1.        , 0.92489701],
       [0.92489701, 1.        ]])

In [17]:
np.corrcoef(res3, Y)

array([[1.        , 0.91623797],
       [0.91623797, 1.        ]])

In [ ]:
# 숙제
- iris.csv 데이터를 로딩한 다음 분류망을 구성하시오
- parameter tuning을 구현하시오(pipeline도 함꼐 사용하시오)
- 일요일 저녁까지